In [214]:
import requests
import re
import pandas as pd
import matplotlib as plt
from datetime import datetime
import numpy as np

In [215]:
file = open('./.env')
api = file.read()

In [216]:
def search_igdb(n): 
  
    #set up an empty list to append each API call
    output = []
    url = 'https://api-v3.igdb.com/games/?limit=50'
    #establish URL and api_key
    headers = {
    'user-key': api,
    'accept': 'application/json'
     }
    #loops through a list of n length and updates offset after each call
    for i in list(range(n)): 
        offset = 0
        offset += 50*i 
        url_params = {
            'fields':'name, total_rating_count, total_rating, rating_count, rating, popularity, aggregated_rating, aggregated_rating_count, themes, platforms, first_release_date, player_perspectives, hypes, game_modes, game_engines, genres', 
            #'filter[release_dates.platform][any]': '48,49,6',
            'filter[total_rating_count][gte]': '150',
            'filter[rating_count][gte]': '100',
            'filter[aggregated_rating_count][gte]': '10',
            'offset': offset
      
      
      
    #         'filter'= filter[release_dates.platform][any] = 48,49,6
    #         'filter'= filter[total_rating_count][gte] = 1
          }
        #fields=*&filter[release_dates.platform][any]=48,49,6
    
    
        response = requests.get(url, headers=headers, params=url_params)
        #run our request get requests
    
  
    
        list(filter(lambda x: output.append(x), response.json()))
     #and 
  
    return output


In [217]:
video_games = search_igdb(4)
df = pd.DataFrame(video_games)

In [218]:
# sort the games by the highest rating 
df = df.sort_values('total_rating', ascending=False)

In [219]:
# change the index of the DataFrame to the name of the game
df = df.set_index('name')

In [220]:
# drop the column with the game 'IDs'
df = df.drop(['id'], axis=1)

In [224]:
# changing column names
df = df.rename(columns={'aggregated_rating':'critic_review_rating'})
df = df.rename(columns={'aggregated_rating_count':'critic_review_rating_count'})
df = df.rename(columns={'first_release_date':'release_date'})
df = df.rename(columns={'rating':'player_review_rating'})
df = df.rename(columns={'rating_count':'player_review_rating_count'})
df = df.rename(columns={'total_rating':'total_review_rating'})
df = df.rename(columns={'total_rating_count':'total_review_rating_count'})

In [247]:
# replace the values in our two rows with 
for row in df.loc[df.themes.isnull(), 'themes'].index: 
    df.at[row, 'themes'] = [] 
        
for row in df.loc[df.player_perspectives.isnull(), 'player_perspectives'].index: 
    df.at[row, 'player_perspectives'] = [] 
    
df = df.fillna(' ')
# df

In [248]:
# getting codes for each column 

platforms_codes = list(df['platforms'])
game_engines_codes = list(df['game_engines'])
player_perspectives_codes = list(df['player_perspectives'])
game_modes_codes = list(df['game_modes'])
themes_codes = list(df['themes'])

def unique_column_set(column):
    unique_letters = [] 
    for letters in column:
        for letter in letters:
            if letters != '':
                unique_letters.append(letter)
    return set(unique_letters)

unique_platforms_codes = unique_column_set(platforms_codes)
unique_game_engines_codes = unique_column_set(game_engines_codes)
unique_player_perspectives_codes = unique_column_set(player_perspectives_codes)
unique_game_modes_codes = unique_column_set(game_modes_codes)
unique_themes_codes = unique_column_set(themes_codes)

In [249]:
# getting all the values for release_dates in a unique set

unix_codes = list(df['release_date'])

def unique_release_set(lst):
    unique_dates = [] 
    for date in lst:
                unique_dates.append(date)
    return set(unique_dates)

unique_release_codes = unique_release_set(unix_codes)
len(unique_release_codes)

171

In [250]:
# converting epoch to human readable date
df['release_date'] = pd.to_datetime(df['release_date'], unit='s')

In [251]:
# creating a new column called year which displays just the year the game released in standart date time
df['release_year'] = df['release_date'].dt.year

In [252]:
# created dictionary with keys as game_mode and values as the corresponding ID
game_modes = {'Single Player':1, 
              'Multiplayer':2 , 
              'Co-operative':3, 
              'Split Screen':4, 
              'Massively Multiplayer Online':5}  

In [253]:
df.head()

,critic_review_rating,critic_review_count,release_date,game_engines,game_modes,genres,hypes,platforms,player_perspectives,popularity,...,Business,Drama,Non-Fiction,Sandbox,Educational,Kids,Open-world,Warfare,Erotic,Mystery
name,,,,,,,,,,,,,,,,,,,,,
Red Dead Redemption 2,95.916667,22,2018-10-26,[26],"[1, 2, 3]","[5, 12, 31]",257,"[48, 49]",[2],224.522804,...,False,False,False,False,False,False,True,False,False,False
God of War,96.944444,21,2018-04-20,,[1],"[12, 31]",92,[48],[2],117.296647,...,False,False,False,False,False,False,False,False,False,False
The Last of Us Remastered,97.400000,11,2014-07-29,,"[1, 2]","[5, 31]",5,[48],[2],23.352280,...,False,False,False,False,False,False,False,False,False,False
The Legend of Zelda: Breath of the Wild,97.592593,31,2017-03-03,"[17, 502, 503]",[1],"[12, 31]",142,"[41, 130]",[2],67.085003,...,False,False,False,False,False,False,True,False,False,False
Super Mario Odyssey,96.705882,19,2017-10-27,,"[1, 3]","[8, 31]",88,[130],[2],37.605908,...,False,False,False,True,False,False,True,False,False,False


In [254]:
#goes through our series function game_modes and creates a new set of columns that passes a boolean True if that game supports that game mode
for game_mode in game_modes.keys():
    df[game_mode] = df.game_modes.apply(lambda mode_list: game_modes[game_mode] in mode_list)
    

    
#got rid of the column with game modes now that we have our game_mode with dummy variable 
# df = df.drop(['game_modes'], axis=1) 
    

In [255]:
#created dictionary with keys as platform and values as corresponding ID for each platform
platforms = {'Linux': 3,
'Wii': 5 ,
'PC': 6,
'PS3': 9,
'Xbox': 11,
'Xbox 360':12,
'Mac': 14,
'Nintendo DS':20,
'Android':34,
'Xbox Live Arcade':36,
'Nintendo 3DS': 37,
'iOS':39,
'Wii U':41,
'PlayStation Network':45,
'PlayStation Vita':46,
'PS4':48,
'Xbox One':49,
'Arcade':52,
'Ouya':72,
'Windows Phone':74,
'SteamOS':92,
'Nintendo Switch':130,
'Nintendo DSi':159}

In [256]:
#goes through each list in the platform column and creates a dummy variable for each platform type and adds a boolean expression if the game in question is offered on that platform
for platform in platforms.keys():
    df[platform] = df.platforms.apply(lambda plat_list: platforms[platform] in plat_list) 
    
# get rid of the old axis with the platforms as list
# df = df.drop(['platforms'], axis=1) 


In [257]:
df.iloc[:,:]

,critic_review_rating,critic_review_count,release_date,game_engines,game_modes,genres,hypes,platforms,player_perspectives,popularity,...,Business,Drama,Non-Fiction,Sandbox,Educational,Kids,Open-world,Warfare,Erotic,Mystery
name,,,,,,,,,,,,,,,,,,,,,
Red Dead Redemption 2,95.916667,22,2018-10-26,[26],"[1, 2, 3]","[5, 12, 31]",257,"[48, 49]",[2],224.522804,...,False,False,False,False,False,False,True,False,False,False
God of War,96.944444,21,2018-04-20,,[1],"[12, 31]",92,[48],[2],117.296647,...,False,False,False,False,False,False,False,False,False,False
The Last of Us Remastered,97.400000,11,2014-07-29,,"[1, 2]","[5, 31]",5,[48],[2],23.352280,...,False,False,False,False,False,False,False,False,False,False
The Legend of Zelda: Breath of the Wild,97.592593,31,2017-03-03,"[17, 502, 503]",[1],"[12, 31]",142,"[41, 130]",[2],67.085003,...,False,False,False,False,False,False,True,False,False,False
Super Mario Odyssey,96.705882,19,2017-10-27,,"[1, 3]","[8, 31]",88,[130],[2],37.605908,...,False,False,False,True,False,False,True,False,False,False
Persona 5,93.571429,23,2016-09-15,,[1],"[8, 12, 31]",34,"[9, 48]",[2],51.211644,...,False,False,False,False,False,False,False,False,False,False
Mass Effect 2,95.600000,14,2010-01-26,[351],[1],"[5, 12, 13]",1,"[6, 9, 12]",[2],26.915687,...,False,False,False,False,False,False,False,False,False,False
The Witcher 3: Wild Hunt,91.720000,26,2015-05-19,[24],[1],"[12, 31]",179,"[6, 48, 49]",[2],111.789564,...,False,False,False,False,False,False,True,False,False,False
Grand Theft Auto V,95.840000,34,2013-09-17,[26],"[1, 2]","[5, 10, 14, 31]",,"[6, 9, 12, 48, 49]","[1, 2]",87.169661,...,False,False,False,True,False,False,True,False,False,False


In [259]:
themes = {'Action':1,
    'Fantasy': 17,
    'Science-Fiction': 18,
    'Horror': 19,
    'Thriller': 20,
    'Survival': 21,
    'Historical': 22,
    'Stealth': 23,
    'Comedy': 27,
    'Business': 28,
    'Drama': 31,
    'Non-Fiction': 32,
    'Sandbox': 33,
    'Educational': 34,
    'Kids': 35,
    'Open-world': 38,
    'Warfare': 39,
    'Erotic': 42,
    'Mystery': 43 }

In [260]:
type(df.themes[0][0])

int

In [261]:
themes['Action'] in df['themes'][0]

True

In [262]:
# for i in df['themes']:
#     try:
#         for j in df['themes'][i]:
#             try:
#                 print(type(j))
#             except:
#                 print(im)
#                 break
#     except:
#         print(type(i))
#         break

In [265]:
for row in df.iterrows():
    print(row)
#    if type(row['themes'][0]) != list:
#        print(type(row['themes']))

('Red Dead Redemption 2', critic_review_rating                        95.9167
critic_review_count                              22
release_date                    2018-10-26 00:00:00
game_engines                                   [26]
game_modes                                [1, 2, 3]
genres                                  [5, 12, 31]
hypes                                           257
platforms                                  [48, 49]
player_perspectives                             [2]
popularity                                  224.523
player_review_rating                        97.4577
player_review_rating_count                      205
themes                                      [1, 38]
total_review_rating                         96.6872
total_review_rating_count                       227
release_year                                   2018
Single Player                                  True
Multiplayer                                    True
Co-operative                          

Name: Max Payne 3, Length: 63, dtype: object)
('XCOM: Enemy Unknown', critic_review_rating                            88.5
critic_review_count                               14
release_date                     2012-10-09 00:00:00
game_engines                                     [6]
game_modes                                    [1, 2]
genres                          [11, 12, 15, 16, 24]
hypes                                               
platforms                          [3, 6, 9, 12, 14]
player_perspectives                              [3]
popularity                                   5.85919
player_review_rating                         83.8181
player_review_rating_count                       217
themes                                   [1, 18, 39]
total_review_rating                          86.1591
total_review_rating_count                        231
release_year                                    2012
Single Player                                   True
Multiplayer                  

Name: Firewatch, Length: 63, dtype: object)
("Assassin's Creed: Revelations", critic_review_rating                            81.7143
critic_review_count                                  10
release_date                        2011-11-14 00:00:00
game_engines                                     [8, 9]
game_modes                                       [1, 2]
genres                                         [16, 31]
hypes                                                  
platforms                                    [6, 9, 12]
player_perspectives                                 [2]
popularity                                      11.6902
player_review_rating                            77.6219
player_review_rating_count                          451
themes                          [1, 17, 22, 23, 33, 38]
total_review_rating                             79.6681
total_review_rating_count                           461
release_year                                       2011
Single Player             

In [264]:
#perform the repeated for loop on the them column and then drop the theme column
for theme in themes.keys():
    df[theme] = df.themes.apply(lambda theme_list: themes[theme] in theme_list) 

#df = df.drop(['themes'], axis=1) 

In [126]:
#checking to make sure that the dataframe has updated both the dummy variables and the drop column
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, Red Dead Redemption 2 to Pokémon GO
Data columns (total 60 columns):
critic_review_rating            183 non-null float64
critic_review_count             183 non-null int64
release_date                    183 non-null datetime64[ns]
game_engines                    183 non-null object
genres                          183 non-null object
hypes                           183 non-null object
player_perspectives             183 non-null object
popularity                      183 non-null float64
player_review_rating            183 non-null float64
player_review_rating_count      183 non-null int64
total_review_rating             183 non-null float64
total_review_rating_count       183 non-null int64
release_year                    183 non-null int64
Single Player                   183 non-null bool
Multiplayer                     183 non-null bool
Co-operative                    183 non-null bool
Split Screen                    183 non

In [266]:
#repeat for player_perspectives
player_perspectives = {'First-Person':1,
    'Third-Person': 2,
    'Bird-view': 3,
    'Side-view': 4,
    'Text': 5,
    'Aural': 6,
    'Virtual Reality': 7}

In [267]:
#repeat for player perspectives
for player_perspective in player_perspectives.keys():
    df[player_perspective] = df.player_perspectives.apply(lambda pp_list: player_perspectives[player_perspective] in pp_list) 

In [280]:
df.loc[:,'First-Person':'Aural'] 

,First-Person,Third-Person,Bird-view,Side-view,Text,Aural
name,,,,,,
Red Dead Redemption 2,False,True,False,False,False,False
God of War,False,True,False,False,False,False
The Last of Us Remastered,False,True,False,False,False,False
The Legend of Zelda: Breath of the Wild,False,True,False,False,False,False
Super Mario Odyssey,False,True,False,False,False,False
Persona 5,False,True,False,False,False,False
Mass Effect 2,False,True,False,False,False,False
The Witcher 3: Wild Hunt,False,True,False,False,False,False
Grand Theft Auto V,True,True,False,False,False,False


In [269]:
df.columns

Index(['critic_review_rating', 'critic_review_count', 'release_date',
       'game_engines', 'game_modes', 'genres', 'hypes', 'platforms',
       'player_perspectives', 'popularity', 'player_review_rating',
       'player_review_rating_count', 'themes', 'total_review_rating',
       'total_review_rating_count', 'release_year', 'Single Player',
       'Multiplayer', 'Co-operative', 'Split Screen',
       'Massively Multiplayer Online', 'Linux', 'Wii', 'PC', 'PS3', 'Xbox',
       'Xbox 360', 'Mac', 'Nintendo DS', 'Android', 'Xbox Live Arcade',
       'Nintendo 3DS', 'iOS', 'Wii U', 'PlayStation Network',
       'PlayStation Vita', 'PS4', 'Xbox One', 'Arcade', 'Ouya',
       'Windows Phone', 'SteamOS', 'Nintendo Switch', 'Nintendo DSi', 'Action',
       'Fantasy', 'Science-Fiction', 'Horror', 'Thriller', 'Survival',
       'Historical', 'Stealth', 'Comedy', 'Business', 'Drama', 'Non-Fiction',
       'Sandbox', 'Educational', 'Kids', 'Open-world', 'Warfare', 'Erotic',
       'Mystery', 'Fi

In [271]:
#repeating the process for genres defining our dictionary
genres = {'Simulator':13,
'Tactical':24,
'Quiz/Trivia':26,
'Fighting':4,
'Strategy':15,
'Adventure':31,
'Role-playing (RPG)':12,
'Shooter':5,
'Music':7,
'Indie':32,
'Turn-based strategy (TBS)':16,
'Pinball':30,
'Puzzle':9,
'Real Time Strategy (RTS)':11,
'Hack and slash/Beat ’em up':25,
'Platform':8,
'Racing':10,
'Sport':14,
'Arcade':33,
'Point-and-click':2}

In [272]:
#ran our for loop for genres
for genre in genres.keys():
    df[genre] = df.genres.apply(lambda genre_list: genres[genre] in genre_list) 

In [276]:
df.iloc[:, -20:-1]

,Virtual Reality,Simulator,Tactical,Quiz/Trivia,Fighting,Strategy,Adventure,Role-playing (RPG),Shooter,Music,Indie,Turn-based strategy (TBS),Pinball,Puzzle,Real Time Strategy (RTS),Hack and slash/Beat ’em up,Platform,Racing,Sport
name,,,,,,,,,,,,,,,,,,,
Red Dead Redemption 2,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False
God of War,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
The Last of Us Remastered,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False
The Legend of Zelda: Breath of the Wild,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
Super Mario Odyssey,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
Persona 5,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False
Mass Effect 2,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
The Witcher 3: Wild Hunt,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
Grand Theft Auto V,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,True


In [291]:
df = df.drop(['platforms'], axis=1) 

In [286]:
df = df.drop(['genres'], axis=1)  

In [287]:
df = df.drop(['game_modes'], axis=1) 

In [288]:
df = df.drop(['player_perspectives'], axis=1) 

In [289]:
df = df.drop(['themes'], axis=1)

In [292]:
df.head()

,critic_review_rating,critic_review_count,release_date,game_engines,hypes,popularity,player_review_rating,player_review_rating_count,total_review_rating,total_review_rating_count,...,Indie,Turn-based strategy (TBS),Pinball,Puzzle,Real Time Strategy (RTS),Hack and slash/Beat ’em up,Platform,Racing,Sport,Point-and-click
name,,,,,,,,,,,,,,,,,,,,,
Red Dead Redemption 2,95.916667,22,2018-10-26,[26],257,224.522804,97.457749,205,96.687208,227,...,False,False,False,False,False,False,False,False,False,False
God of War,96.944444,21,2018-04-20,,92,117.296647,96.300186,377,96.622315,398,...,False,False,False,False,False,False,False,False,False,False
The Last of Us Remastered,97.400000,11,2014-07-29,,5,23.352280,93.215751,465,95.307876,476,...,False,False,False,False,False,False,False,False,False,False
The Legend of Zelda: Breath of the Wild,97.592593,31,2017-03-03,"[17, 502, 503]",142,67.085003,92.582298,469,95.087445,500,...,False,False,False,False,False,False,False,False,False,False
Super Mario Odyssey,96.705882,19,2017-10-27,,88,37.605908,91.955860,279,94.330871,298,...,False,False,False,False,False,False,True,False,False,False
